In [1]:
import sys
sys.path.append('./src/')
from functions import *
from models import *
#
import matplotlib.pyplot as plt
#
from tensorboardX import SummaryWriter  
#
import argparse


Set all parser arguments.

In [ ]:

parser = argparse.ArgumentParser()

# Mode settings
parser.add_argument('--train_mode', default=True, type=bool) # Set to False for Test mode.
parser.add_argument('--load_model', default=False, type=bool)

# Classes settings
parser.add_argument('--dist_cut', default=10.0, type=float) # distance cut-off for class defintion
parser.add_argument('--N_classes', default=2, type=int) # Number of classes
parser.add_argument('--desired_labels', default=[0,1], type=list) # Classes to be considered for output

# Bio-system settings
parser.add_argument('--biosystem', default='PROTEIN', type=str) # Necessary for atom selections (see below)

# Model settings
parser.add_argument('--n_epochs', default=1000, type=int) # Number of epochs for training
parser.add_argument('--batch_size', default=100, type=int) # Batch size for training
parser.add_argument('--learning_rate', default=0.0002, type=float) # Learning rate for Adam optimizer
parser.add_argument('--noise_dim', default=100, type=int) # Dimension for gaussian noise to feed to the generator

# Output settings
parser.add_argument('--desired_format', default='inpcrd', type=str)
parser.add_argument('--epoch_freq', default=10, type=int) # Output will be generated every this many epochs
parser.add_argument('--log_freq', default=100, type=int) # Verbose output will be printed every this many epochs
parser.add_argument('--n_structures', default=10, type=int) # Number of structures to be generated for every class (only in Test mode)
parser.add_argument('--input_directory', default='./example_input/' , type=str) 
parser.add_argument('--output_directory', default='./example_output/', type=str) 

args = parser.parse_args()

prmf = args.input_directory+'peptide.prmtop' # Parameter and topology file
#trajfs = [args.input_directory+'all_conformations.mdcrd'] # Trajectory files list
trajfs = ['/home/acrnjar/Desktop/TEMP/Peptides_gen/all_conformations.mdcrd'] # Trajectory files list

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Cuda is available:",torch.cuda.is_available())

### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###

def main():

    print("Output will be written in:",args.output_directory)
    if not os.path.exists(args.output_directory): os.system('mkdir '+args.output_directory)

    if args.biosystem=='PROTEIN':
        backbone='name CA C N'
    elif args.biosystem=='DNA':
        backbone='name P'

    print("train_mode:",args.train_mode)
    print("load_model:",args.load_model)

    model_g_file=args.output_directory+'model_generator.pth'
    model_d_file=args.output_directory+'model_discriminator.pth'

    # Remove previous output if necessary
    if args.train_mode:
        os.system('rm '+args.output_directory+'gen*'+args.desired_format+' 2> /dev/null')
        if not args.load_model:
            os.system('rm '+args.output_directory+'out*'+args.desired_format+' 2> /dev/null')

    # Determine last saved epoch   
    last_epoch=0
    if args.train_mode and args.load_model:
        prefixed = [filename for filename in os.listdir(args.output_directory) if filename.startswith("out_train_label1_epoch")]
        past_epochs=[]
        for wfile in prefixed:
            past_epochs.append(int(wfile.replace('out_train_label1_epoch','').replace('.'+desired_format,'')))
        last_epoch=max(past_epochs)
        print("Last epoch found:",last_epoch)

    # Define MDAnalysis universe and related parameters
    univ = mda.Universe(prmf, trajfs)
    nframes = len(univ.trajectory)
    batch_freq = int(nframes/args.batch_size) 
    N_at = len(univ.select_atoms('all'))
    box_s = max_size(prmf,trajfs,'all',1.1) # Calculate largest coordinate for generation

    # Generate data
    dataset,atoms_list = generate_training_data(prmf, trajfs, 0, nframes-(nframes%args.batch_size), backbone, args.dist_cut, args.output_directory)

    # Define discriminator and generator models
    discriminator = DiscriminatorModel(N_at, args.N_classes, n1=50, n2=100, n3=200) 
    generator = GeneratorModel(N_at, args.noise_dim, args.N_classes, box_s, n1=200, n2=100, n3=50) 
    discriminator.to(device)
    generator.to(device)
    if (args.load_model==True and os.path.isfile(model_g_file) and os.path.isfile(model_d_file)):
        print("{} and {} loaded.".format(model_g_file,model_d_file))
        generator.load_state_dict(torch.load(model_g_file))
        discriminator.load_state_dict(torch.load(model_d_file))

    # For a conditional GAN, the loss function is the Binary Cross Entropy between the target and the input probabilities
    loss = nn.BCELoss() 

    # Define optimizers
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=args.learning_rate) 
    generator_optimizer = optim.Adam(generator.parameters(), lr=args.learning_rate)

    # Load data
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=True)

    if args.train_mode:
        summary_writer = SummaryWriter(args.output_directory)
        
        # initialize lists for observables and graphs.
        e2e_distance = []
        bonds_dev = []
        angles_dev = []
        losses_fig = plt.figure(1, figsize=(4, 4))
        e2e_fig = plt.figure(1, figsize=(4, 4))
        bonds_fig = []
        angles_fig = []
        for dl, d_label in enumerate(args.desired_labels):
            e2e_distance.append([])
            bonds_dev.append([])
            angles_dev.append([])
            bonds_fig.append(plt.figure(1, figsize=(4, 4)))
            angles_fig.append(plt.figure(1, figsize=(4, 4)))
        
        Loss_G_mean = []
        Loss_D_mean = []
        for epoch_idx in range(args.n_epochs): 
            G_loss = [] 
            D_loss = []    
            for batch_idx, data_input in enumerate(data_loader):
            
                # Generate noise and move it the device
                noise = torch.randn(args.batch_size,args.noise_dim).to(device) 
                # Forward pass
                fake_labels = torch.randint(0,args.N_classes,(args.batch_size,)).to(device)
                generated_data = generator(noise, fake_labels) 
                
                true_data = data_input[0].view(args.batch_size, 3*N_at).to(device) 
                digit_labels = data_input[1].to(device) 
                true_labels = torch.ones(args.batch_size).to(device) 

                # Clear optimizer gradients        
                discriminator_optimizer.zero_grad()
                # Forward pass with true data as input
                discriminator_output_for_true_data = discriminator(true_data,digit_labels).view(args.batch_size) 
                # Compute Loss
                true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels) 
                
                # Forward pass with generated data as input
                discriminator_output_for_generated_data = discriminator(generated_data.detach(), fake_labels).view(args.batch_size) 
                # Compute Loss
                generator_discriminator_loss = loss(discriminator_output_for_generated_data, torch.zeros(args.batch_size).to(device)) 
                # Average the loss
                discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2 
                # Backpropagate the losses for Discriminator model.
                discriminator_loss.backward()
                discriminator_optimizer.step()
                D_loss.append(discriminator_loss.data.item())
                
                # Clear optimizer gradients
                generator_optimizer.zero_grad()        
                # It's a choice to generate the data again 
                generated_data = generator(noise, fake_labels) #.requires_grad_(False) 
                # Forward pass with the generated data
                discriminator_output_on_generated_data = discriminator(generated_data, fake_labels).view(args.batch_size) 
                # Compute loss: it must be the same of the discriminator, but reversed: the fake data must be passed as all ones, thus we use true_labels
                generator_loss = loss(discriminator_output_on_generated_data, true_labels) 
                # Backpropagate losses for Generator model.
                generator_loss.backward()
                generator_optimizer.step()
                G_loss.append(generator_loss.data.item())
                
                if (batch_idx==0 and epoch_idx==0): print("Initial: discriminator_loss: {} , generator_loss: {}".format(discriminator_loss.item(),generator_loss.item()))

                # Evaluate the model
                if ((batch_idx + 1)% batch_freq == 0 and (epoch_idx + 1)%args.epoch_freq == 0): 
                    with torch.no_grad(): 
                        noise = torch.randn(args.batch_size,args.noise_dim).to(device)
                        for dl, d_label in enumerate(args.desired_labels):
                            fake_labels = torch.tensor(args.batch_size*[dl]).to(device) 
                            generated_data = generator(noise, fake_labels).cpu().view(args.batch_size, 3*N_at) 
                            for x in generated_data:

                                # Generate .inpcrd file
                                outname='out_label'+str(fake_labels[0].item())+'_epoch'+str(last_epoch+epoch_idx+1)+'.inpcrd'
                                write_inpcrd(x.detach().numpy().reshape(N_at,3),outname=args.output_directory+outname)
                                if (epoch_idx+1)%args.log_freq==0: print("{} written.".format(outname))

                                # Calculate observables for later evaluation of the training
                                e2e_distance[dl].append([epoch_idx,check_label_condition(prmf,args.output_directory+outname)])
                                bonds_dev[dl].append([epoch_idx,bonds_deviation(prmf,args.output_directory+outname)])
                                angles_dev[dl].append([epoch_idx,angles_deviation(prmf,args.output_directory+outname)]) 
                                torch.save(generator.state_dict(),model_g_file) 
                                torch.save(discriminator.state_dict(),model_d_file) 
                                break
                    
            if (epoch_idx+1)%args.epoch_freq==0:
                summary_writer.add_scalar('Loss_d',torch.mean(torch.FloatTensor(D_loss)),global_step=epoch_idx)
                summary_writer.add_scalar('Loss_g',torch.mean(torch.FloatTensor(G_loss)),global_step=epoch_idx)
                Loss_D_mean.append([epoch_idx,torch.mean(torch.FloatTensor(D_loss))])
                Loss_G_mean.append([epoch_idx,torch.mean(torch.FloatTensor(G_loss))])

            if (epoch_idx+1)%args.log_freq==0:
                print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % ( (epoch_idx+last_epoch+1), args.n_epochs+last_epoch, torch.mean(torch.FloatTensor(D_loss)), torch.mean(torch.FloatTensor(G_loss))))

        # Plot loss averages over batches
        plt.plot(np.array(Loss_D_mean)[:, 0], np.array(Loss_D_mean)[:, 1],lw=1,c='C0',label='Discriminator')
        plt.plot(np.array(Loss_G_mean)[:, 0], np.array(Loss_G_mean)[:, 1],lw=1,c='C1',label='Generator')
        plt.legend(loc='upper right',prop={'size':15})
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        losses_fig.savefig(args.output_directory+'Losses.png',dpi=150)
        #plt.show()
        plt.clf()

        # Plot observables
        for dl, d_label in enumerate(args.desired_labels):
            plt.plot(np.array(bonds_dev[dl])[:, 0], np.array(bonds_dev[dl])[:, 1],lw=1,c='C0')
            plt.xlabel('Epoch')
            plt.ylabel('Bonds dev. [$\AA$]')
            bonds_fig[dl].savefig(args.output_directory+'Bonds_deviation_label'+str(d_label)+'.png',dpi=150)
            #plt.show()
            plt.clf()
            plt.plot(np.array(angles_dev[dl])[:, 0], np.array(angles_dev[dl])[:, 1],lw=1,c='C1')
            plt.xlabel('Epoch')
            plt.ylabel('Angle dev. [deg]')
            angles_fig[dl].savefig(args.output_directory+'Angles_deviation_label'+str(d_label)+'.png',dpi=150)
            #plt.show()
            plt.clf()

        # Plot the end-to-end distances
        for dl, d_label in enumerate(args.desired_labels):
            plt.plot(np.array(e2e_distance[dl])[:, 0], np.array(e2e_distance[dl])[:, 1],lw=1,c='C'+str(dl),label='Label '+str(d_label))
        plt.xlabel('Epoch')
        plt.ylabel('End-to-end distance [$\AA$]')
        plt.legend(loc='upper right',prop={'size':15})
        e2e_fig.savefig(args.output_directory+'End2end_distances.png',dpi=150)
        #plt.show()
        plt.clf()

    # Test mode
    else: 
        for structure_idx in range(args.n_structures):
            print("Generating structure:",structure_idx)
            with torch.no_grad(): 
                noise = torch.randn(args.batch_size,args.noise_dim).to(device)
                for dl,d_label in enumerate(args.desired_labels):
                    fake_labels = torch.tensor(args.batch_size*[dl]).to(device) 
                    generated_data = generator(noise, fake_labels).cpu().view(args.batch_size, 3*N_at) 
                    for x in generated_data:
                        outname='gen_label'+str(fake_labels[0].item())+'_'+str(structure_idx+1)+'.inpcrd' 
                        write_inpcrd(x.detach().numpy().reshape(N_at,3),outname=args.output_directory+outname)
                        print("{} written.".format(outname))
                        break
        

if __name__ == '__main__':
    main()




